In [2]:
!pip install PyPDF2

You are using pip version 9.0.3, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [40]:
!pip install textract

    100% |████████████████████████████████| 8.9MB 105kB/s eta 0:00:01
    100% |████████████████████████████████| 32.8MB 27kB/s  eta 0:00:01
    100% |████████████████████████████████| 112kB 3.8MB/s eta 0:00:01
    100% |████████████████████████████████| 102kB 3.7MB/s ta 0:00:01
    100% |████████████████████████████████| 5.6MB 219kB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 2.4MB/s eta 0:00:01
    100% |████████████████████████████████| 112kB 3.6MB/s eta 0:00:01
    100% |████████████████████████████████| 143kB 3.2MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/23/8c/22a47a4bf8c5289e4ed946d2b0e4df62bca385b9599cc1e46878f2e2529c/sortedcontainers-2.2.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/af/16/da16a22d47bac9bf9db39f3b9af74e8eeed8855c0df96be20b580ef92fff/pycryptodome-3.9.7-cp36-cp36m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 81kB 3.3MB/s eta 0:00:01
  Running setup.py bdist_wheel f

In [56]:
import textract
text = textract.process("Sisseton Wahpeton Community College2019_2021.pdf")

In [70]:
text.decode('utf-8').replace('\n', '').split('\x0c')[69]

' TR 180, TR 280 Special Topics  Significant issues or consideration of particular technical aspects of building trades are possible investigations. (1-3)  TR 190, TR 290 Independent Study  Advanced study of one or more selected topics according to students’ special needs and interests in the area of building trades. A course not listed as a regular offering in the catalog. Requires objectives, scope of work and plan of study specified by instructor and/or student. Consent of advisor and VPAA. May be taken more than one time. (1-3)  TR 295 General Building Trades Internship Supervised experience in a General Building Trades related position. Students can choose the setting for internship. Internship specifics will be determined by setting supervisor and instructor. (3)  EL 101 Fundamentals of Electrical Construction  This course is currently under development.  MA 101 Fundamentals of Concrete Masonry  This course will introduce the fundamentals of masonry trade. History, tools, equipme

In [3]:
import PyPDF2 as pypdf
import regex as re
import numpy as np
import csv
import pandas as pd

In [64]:
PDFfile = open("Sisseton Wahpeton Community College2019_2021.pdf", "rb")
pdfread = pypdf.PdfFileReader(PDFfile)
pdfread

In [65]:
y = pdfread.getPage(51)
y.extractText().replace('\n', '')

'  '

In [5]:
pdfread.getNumPages()

285

In [19]:
def get_data(csv_name, pdf_name, page_range, pattern, dep_group, abbrev_group, c_group, t_group, desc_group,
            year, grad_status):
    """Takes in name of PDF in local folder and extracts data in given PAGE_RANGE 
    using regex expression PATTERN and given group numbers. YEAR is the catalog year.
    GRAD_STATUS is either None (if unknown), 'G', 'UG', or a number indicating at
    which number is the course graduate."""
    
    PDFfile = open(pdf_name, "rb")
    pdfread = pypdf.PdfFileReader(PDFfile)
    
    csv_writer = csv.writer(open(csv_name, 'w'))
    csv_writer.writerow(["Department Name", "Course Catalogue Number", "Course Name",
     "Course Description", "Graduate/Undergraduate", "Format", "Lab", 
     "Academic Catalogue Year"])
    FS_keys = ["agri", "food", "animal"]
    
    look_ahead = pattern + r'(.+?)(?=' + pattern + r')'
    
    dep_names = {}
    #test = page_range[0] #DEBUGGING
    text = ''
    for i in np.arange(page_range[0], page_range[1]):
    #    print(test) #DEBUGGING
    #    test += 1 #DEBUGGING
        text += pdfread.getPage(i).extractText().replace('\n', ' ')    
    matches = re.findall(look_ahead, text)
    last = re.sub(look_ahead, '', text)
    last = re.search(pattern + r'(.+)', last)
    matches = matches + [last.groups()]

    for match in matches:  
        #print(match) #DEBUGGING
        # Getting full department name
        print(dep_names)
        if not dep_names.get(match[abbrev_group - 1]):
            dep_names[match[abbrev_group - 1]] = match[dep_group - 1]
        dep_name = dep_names.get(match[abbrev_group - 1])

        # Other groups
        course_num = int(match[c_group - 1])
        title = str(match[t_group - 1])
        desc = str(match[desc_group - 1]) # assuming description is group 5

        #Filtering
        #for key in FS_keys:
        #    if (re.search(key, title, flags=re.IGNORECASE) or 
        #        re.search(key, desc, flags=re.IGNORECASE)):
        G_UG = grad_status
        if grad_status:
            if type(grad_status) == int:
                G_UG = "UG"
                if int(t.group(1)) >= grad_status:
                    G_UG = "G"
        lab = False
        if (re.search(r'Laboratory|Lab', title, flags=re.IGNORECASE) or 
        re.search(r'Laboratory|Lab', desc, flags=re.IGNORECASE)):
            lab = True
        csv_writer.writerow([dep_name, course_num, title, desc, 
                                 G_UG, None, lab, year])
                                # Leaving format blank for now
                        
    print(dep_names)
                
pattern = r'(([A-Z]+)[A-Z\s]+\s)?([A-Z]+)\s(\d+)(.+?)(\sPre\s-requisite.+?)?\d\s{0,2}credits'
#Partial Test set:
get_data('OglalaLakota_sample.csv', "OglalaLakota2019-2020.pdf", [98, 99], pattern, 
         2, 3, 4, 5, 7, '2019-2020', None)

{}
{'ECH': 'EDUCATION'}
{'ECH': 'EDUCATION'}
{'ECH': 'EDUCATION'}
{'ECH': 'EDUCATION'}
{'ECH': 'EDUCATION'}
{'ECH': 'EDUCATION'}
{'ECH': 'EDUCATION'}


In [20]:
pd.read_csv('OglalaLakota_sample.csv')

,Department Name,Course Catalogue Number,Course Name,Course Description,Graduate/Undergraduate,Format,Lab,Academic Catalogue Year
0,EDUCATION,203,Introduction toEarly Childhood Education,. This course provides anintroduction tothe fi...,NaN,NaN,False,2019-2020
1,EDUCATION,213,Planning andAdministrating Early Childhood Pro...,. This course introduces the students tothe st...,NaN,NaN,False,2019-2020
2,EDUCATION,223,Materials &Techniques for Infant/Toddler/Pre -...,. This course will help the candidate todevelo...,NaN,NaN,False,2019-2020
3,EDUCATION,233,Curriculum for Self -Awareness andIndividual D...,. This course isdesigned toprovide candidates ...,NaN,NaN,False,2019-2020
4,EDUCATION,243,Early Childhood Specialty Internship,. Candidates will complete atotal of135hours o...,NaN,NaN,False,2019-2020
5,EDUCATION,253,Parental Staff and Community Involvement inEar...,. This course introduces the candidate tothe i...,NaN,NaN,False,2019-2020
6,EDUCATION,323,"Materials &Techniques IIfor Infant, Toddler, ...",. This course will help candidates todevelop m...,NaN,NaN,False,2019-2020


In [13]:
sample = 'HCA 2513                                           3 Hours Community Health A course designed to create an awareness of current trends, basic issues, controversial issues, and fundamental principles in Health Promotion and Health Education. Particular emphasis will be placed in promoting the physical, mental, and social welfare of citizens in schools and communities based on The Healthy People 2020: Health Promotion and Disease Prevention Objectives. HCA 2523                                            3 Hours Personal and Family Living The course addresses the personal and family relationships throughout the family life cycle. Students will gain insight into their role in society. Areas of concentration are decision making, communication, marriage, parenting, dealing with family issues from a Native American perspective. HCA 2533 3 Hours Utilization of Community Resources Students will be taught the value of fostering good community relations that directly impact program services. The function of various community services will be assessed as well as the study of those resources to enhance the quality of assistance to clients.'
rest = re.sub(cmn_pattern + r'(.+?)(?=' + cmn_pattern + r')', '',sample)
rest

'HCA 2533 3 Hours Utilization of Community Resources Students will be taught the value of fostering good community relations that directly impact program services. The function of various community services will be assessed as well as the study of those resources to enhance the quality of assistance to clients.'

In [18]:
rest_match = re.search(cmn_pattern + r'(.+)', rest)
print(rest_match.group(2))
print(rest_match.group(3))
print(rest_match.group(4))
print(rest_match.group(5))
print(rest_match.group(7))
rest_match.groups()

None
HCA
2533
Utilization of Community Resources
Students will be taught the value of fostering good community relations that directly impact program services. The function of various community services will be assessed as well as the study of those resources to enhance the quality of assistance to clients.


(None,
 None,
 'HCA',
 '2533',
 'Utilization of Community Resources',
 ' be',
 'Students will be taught the value of fostering good community relations that directly impact program services. The function of various community services will be assessed as well as the study of those resources to enhance the quality of assistance to clients.')

In [15]:
sample = '20     COURSE  PREFIX  CODE,  TITLE AND COURSE DES CRIPTION   ACCOUNTING (ACC)   ACC 100 Fundamentals of Accounting (3)   Prerequisite:     This course is an introduction to accounting fundamentals. It includes journalizing, posting, worksheets,  financial reports, and data processing for accounting. One hour laboratory required.   ACC 200 Financial Accounting (3)      This course covers the theory and practice of accounting in services and merchandising firms. It  emphasizes the accounting cycle, periodic reporting, receivables, payables, inventories, and accounting  systems . One hour laboratory required.   ACC 201 Managerial Accounting (3)      This course is a continuation of ACC 200 with emphasis on manufacturing firms, and managerial  approaches for decision - making. One hour laborato ry required.   ACC 310 Government and Non - Profit   Accounting (3)   Prerequisite: Minimu  .   This course covers the theory and practice of accounting for governmental and not - for - profit units.  Topics include financial reporting; budgetary a ccounting; fund accounts; and nonprofit accounting for  entities such as public colleges, universities and health care providers .   ACC 350 Finance (3)    ACC 310,  BUS 265, ECO 200 or 201.   Introduction to the theories and applications associated with the functions of capital markets. This  includes the conceptual foundations of portfolio theory, risk management, and asset valuation. This  course will examine the markets for equity, money, and  bonds. One hour laboratory required.   AGRICULTURE (AGR)   AGR 103 Conservation Planning (4)   This course introduces students to the understanding natural resource conservation issues and concerns  to allow them to develop conservation plans necessary to improve   natural resource conditions applicable  to tribal lands.   AGR 105 Int roduction to Animal Science (4)   This course will introduce students to animal and livestock industries, production, systems and markets.  Students will not only learn about national product ion but include Navajo Nation livestock industry.   AGR 110 Monitoring/Assessments and Rangeland Management (4)   This course will introduce students to the various grazing management concept, identify local vegetation  communities, identification of local plan ts, as related to current Navajo producers grazing management  systems. Field trips and field works will serve as a lab for hands - on and field experience to learn range  and vegetation monitoring concepts.   '
dine_pattern = r'(\w+\s\(\w{2,3}\))?\s+([a-zA-Z]{2,3})\s([\d]+\w?)\s([^\d]+)\(\d\)\s+(.+?)'
last = re.sub(dine_pattern + '(?=' + dine_pattern + ')', '', sample)
l

'20     COURSE  PREFIX  CODE,  TITLE AND COURSE DES CRIPTION      AGR 110 Monitoring/Assessments and Rangeland Management (4)   This course will introduce students to the various grazing management concept, identify local vegetation  communities, identification of local plan ts, as related to current Navajo producers grazing management  systems. Field trips and field works will serve as a lab for hands - on and field experience to learn range  and vegetation monitoring concepts.   '

In [9]:
r'test' + r'test2'

'testtest2'

In [15]:
dine_pattern + r'(?=' + dine_pattern + r')'

'(\\w+\\s\\(\\w{2,3}\\))?\\s+([a-zA-Z]{2,3})\\s([\\d]+\\w?)\\s([^\\d]+)\\(\\d\\)\\s+(.+?)(?=(\\w+\\s\\(\\w{2,3}\\))?\\s+([a-zA-Z]{2,3})\\s([\\d]+\\w?)\\s([^\\d]+)\\(\\d\\)\\s+(.+?))'

In [33]:
sample
re.findall(dine_pattern + r'(.+)(?=' + dine_pattern + r')', sample)[0]

('ACCOUNTING (ACC)',
 'ACC',
 '100',
 'Fundamentals of Accounting ',
 'Prerequisite:     This course is an introduction to accounting fundamentals. It includes journalizing, posting, worksheets,  financial reports, and data processing for accounting. One hour laboratory required.   ACC 200 Financial Accounting (3)      This course covers the theory and practice of accounting in services and merchandising firms. It  emphasizes the accounting cycle, periodic reporting, receivables, payables, inventories, and accounting  systems . One hour laboratory required.   ACC 201 Managerial Accounting (3)      This course is a continuation of ACC 200 with emphasis on manufacturing firms, and managerial  approaches for decision - making. One hour laborato ry required.   ACC 310 Government and Non - Profit   Accounting (3)   Prerequisite: Minimu  .   This course covers the theory and practice of accounting for governmental and not - for - profit units.  Topics include financial reporting; budgetary a

In [65]:
if not None:
    print(True)

True
